According to the [BETYdb documentation](https://pecan.gitbook.io/betydbdoc-dataentry), 
BETYdb is used to manage and distribute agricultural and ecological data. 
The web interface provides a sequence of pages that walk through the process of entering meta-data, 
and then the option of entering trait and yield data through a similar web form or via upload of a text (csv) file. 
For entering large tables of data there is Bulk Upload Wizard. 
This is useful when entering more than a few dozen trait or yield data from a single source.

There are typically two categories of data that are entered:

- Results from previously published research, typically statistical summaries.
- Primary data, observations at the level of an individual replicate. 

BETYdb supports both of these, because it was designed to support new research that quantitatively builds on previous research, 
allowing researchers to develop, test, and evaluate new hypotheses based on what is already known. 


# Install BetyDB with helm

Run the commands below to install BetyDB on top of PostgreSQL for ecological forecasting data. 

In [ ]:
%%bash
oc delete secret/pecan-betydb
oc create secret generic pecan-betydb \
  --from-literal=postgresqlPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=betyPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=secretKey="thisisnotasecret"
echo DONE

In [ ]:
%%bash
git clone https://github.com/computate-org/bety-helm.git ~/.local/src/bety-helm/
helm repo add bitnami https://raw.githubusercontent.com/bitnami/charts/archive-full-index/bitnami
(cd ~/.local/src/bety-helm/ && helm dependency build)
helm upgrade --install betydb ~/.local/src/bety-helm/  \
  --set image.repository=quay.io/computateorg/bety \
  --set image.tag=latest \
  --set image.pullPolicy=Always \
  --set postgresql.enabled=false \
  --set postgresql.fullnameOverride=pecan-postgresql \
  --set postgresql.serviceAccount.enabled=true \
  --set postgresql.postgresqlHost=pecan-postgresql \
  --set serviceAccount.enabled=true \
  --set serviceAccount.name=pecan-betydb \
  --set externalSecret=true \
  --set nameOverride=pecan \
  --set fullnameOverride=pecan-betydb
echo DONE